In [1]:
import pickle as pkl
import numpy as np
import os.path as osp
root='letters_data/checkpoints/'

In [33]:
with open(osp.join(root,'2','results.pkl'),'rb') as f:
    gen_list = pkl.load(f)

In [34]:
psnrs=[]
for o, data in gen_list:
    for i in range(o.shape[0]):
        #change to 0..255
        x = 127.5 * o[i] + 127.5
        y = data[0][i]
        #mse
        mse = np.sum( np.power(x-y,2) ) / np.prod( x.shape )
        print(mse)
        #psnr
        psnr = 20 * ( np.log10(255) - np.log10(np.sqrt(mse)) )
        psnrs.append(psnr)

9051.50260417
1816.44319661
13352.3333333
9728.10611979
4611.4156901
11542.1744792
7749.05078125
1208.87304688
1668.30045573
7074.28776042
10315.109375
10875.9238281
10246.5273438
7880.02213542
12894.0
4910.23307292
4034.22460938
7761.94466146
5552.66536458
2665.87386068
3428.43229167
7331.49348958
14119.4570312
12486.6731771
1842.61230469
10881.2324219
7584.47526042
4006.91536458
12261.7760417
5986.9921875
9495.40039062
12452.7942708
13456.640625
14212.9309896
3821.79589844
1952.65478516
9449.92057292
4486.59277344
8190.93033854
7704.39973958
6281.69791667
6467.17578125
2303.72444661
7068.03776042
9319.17057292
7375.84635417
2371.00732422
8845.07682292
7662.21875
6065.03776042
3987.51757812
2561.03059896
726.984130859
3582.48242188
8841.56835938
7572.35351562
8637.88151042
6944.85221354
2.9843638738
12732.4869792
12291.4114583
12305.2369792
13065.4739583
1037.50390625
6137.23046875
6494.453125
8804.08203125
5203.53190104
11072.5117188
5739.28645833
11200.1901042
10274.7298177
6971.652

In [35]:
np.mean(psnrs),np.std(psnrs)

(10.307005433673648, 3.9005295778656088)